# Generate files for training and prediction in MVP models.
# raw input is WSGA selected columns

In [ ]:
import csv
import pandas as pd
import pysam
import numpy as np
import sys
import Bio.SubsMat.MatrixInfo

In [2]:
def matrix_score(a_0, a_1, name_matrix="blosum62"):
    """
    Input: a str a_0, a str a_1, a str name_matrix
    Output: The matrix score of a_0 and a_1 in the matrix name_matrix
    """
    # Biopython also included placeholder amino acids
    #(B. J, X, Z in its scoring matrix.
    matrix = getattr(Bio.SubsMat.MatrixInfo, name_matrix)
    # Since PAM250 in Biopython is not symmetric, if (a_0, a_1) does not exist,
    # matrix_score() will check if (a_1, a_0) exists.
    if (a_0, a_1) in matrix:
        return matrix[(a_0, a_1)]
    elif (a_1, a_0) in matrix:
        return matrix[(a_1, a_0)]
    else:
        return -1    
        
FASTA_LOC = '/data/hq2130/large_files/resources/hg19.fasta'
def add_gc_content(info):
    chrom, pos = info['hg19_chr'], int(info['hg19_pos(1-based)'])
    fastafile = pysam.Fastafile(FASTA_LOC)
    seq = fastafile.fetch(chrom, pos - 5, pos + 5).upper()
    gc_count = 0
    for dna in seq:
        if dna in {'G', 'C'}:
            gc_count += 1
    gc_count = gc_count / 10.0
    info['gc_content'] = gc_count
    return info

def add_s_het(info):
    gene = info['genename']
    info['s_het_log'] = -1
    if gene in s_het:
        info['s_het_log'] = np.log(s_het[gene] + 1)  # minimal value of s_het = 0.000206342
    return info

def add_gene_metric(info):
    info['pli'] = pli.get(info['genename'], -1)
    info['lofz'] = lofz.get(info['genename'], -1)
    info['prec'] = prec.get(info['genename'], -1)
    info['domino'] = domino.get(info['genename'], -1)
    return info

def add_target(info, target):
    info['target'] = target_value
    if target_value == 'NA' and 'cancer_target' in info:
        info['target'] = int(info['cancer_target'])  
    elif target_value == 'NA' and 'category' in info:
        if info['category'] == 'TP':
            info['target'] = 1
        elif info['category'] == 'TN':
            info['target'] = 0 
    return info

def add_gnomad(info):
    var_id = info['var_id']
    info['gnomad']= gnomad_af.get(var_id, 0) 
    info['gnomad_exome']= gnomad_exome_af.get(var_id, 0) 
    return info

def add_secondary(info):
    gene, aaref = info['genename'], info['aaref']
    info['secondary_H'] = 0
    info['secondary_C'] = 0
    info['secondary_E'] = 0
    if gene in secondary:
        aapos = info['aapos'].split(';')
        for pos in aapos:
            pos = int(pos)
            # AA_seq start from 0(it's a list)
            protein_length = len(AA_seq[gene])
            if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                if pos in secondary[gene]:
                    if secondary[gene][pos] == 'H':
                        info['secondary_H'] = 1
                    elif secondary[gene][pos] == 'C':
                        info['secondary_C'] = 1    
                    elif secondary[gene][pos] == 'E':
                        info['secondary_E'] = 1
    return info


def add_BioPlex(info):
    ''' some feather related to protein? added all pint 
        http://bioplex.hms.harvard.edu/downloadInteractions.php
    '''
    gene = info['genename']
    info['BioPlex'] = BioPlex.get(gene, 0)
    return info

REVEL = '/data/hq2130/large_files/revel_file/revel_all_chr.txt.gz'  # REVEL loc
f_revel= pysam.TabixFile(REVEL)
def add_REVEL(info):
    info['REVEL'] = -1
    chrom, pos = info['hg19_chr'], info['hg19_pos(1-based)']
    ref, alt = info['ref'], info['alt']
    for row in f_revel.fetch(chrom, int(pos)-1, int(pos)+1):# 0-based inputin .fetch
        row = row.split('\t')
        if row[3] == ref and row[4] == alt:
            info['REVEL'] = row[5]
    return info  

MPC = '/data/hq2130/large_files/fordist_constraint_official_mpc_values.txt.gz'  # mpc
f_MPC = pysam.TabixFile(MPC)
def add_MPC(info):
    info['MPC'] = -1
    info['mis_badness'] = -1
    info['obs_exp'] = -1
    chrom, pos = info['hg19_chr'], info['hg19_pos(1-based)']
    ref, alt = info['ref'], info['alt']
    for row in f_MPC.fetch(chrom, int(pos)-1, int(pos)+1):# 0-based inputin .fetch
        row = row.split('\t')
        if row[2] == ref and row[3] == alt:
            info['MPC'] = row[-1]
            info['mis_badness'] = row[-3]
            info['obs_exp'] = row[-4]
    return info

def add_phospho(info):
    info['phospho_score'] = 0
    info['phospho_cutoff'] = 0
    info['phospho_diff'] = 0
    gene, aaref = info['genename'], info['aaref']
    if gene in phosphorylation:
        aapos =map(int, info['aapos'].split(';'))
        
        new_pos = list(aapos)
        for pos in aapos: # pos is 1 based            
            for i in range(1, 7): # left, right flanking of 7 bases
                new_pos.append(pos + i)
                new_pos.append(pos - i)
        for pos in new_pos:
            if pos in phosphorylation[gene]: #if phosphorylation[gene][pos]['AA'] == aaref:
                info['phospho_score'] = phosphorylation[gene][pos]['Score']
                info['phospho_cutoff'] = phosphorylation[gene][pos]['Cutoff']
                info['phospho_diff'] = phosphorylation[gene][pos]['diff']
                break
    return info
                          
def add_SUMO(info):
    gene, aaref = info['genename'], info['aaref']
    info['SUMO_score'] = 0
    info['SUMO_cutoff'] = 0
    info['SUMO_diff'] = 0
    if gene in SUMO:
        aapos =map(int, info['aapos'].split(';'))
        new_pos = list(aapos)
        for pos in aapos: # pos is 1 based            
            for i in range(1, 7): # left, right flanking of 7 bases
                new_pos.append(pos + i)
                new_pos.append(pos - i)
        for pos in new_pos:
            if pos in SUMO[gene]: #if phosphorylation[gene][pos]['AA'] == aaref:
                info['SUMO_score'] = SUMO[gene][pos]['Score']
                info['SUMO_cutoff'] = SUMO[gene][pos]['Cutoff']
                info['SUMO_diff'] = SUMO[gene][pos]['diff']
                break
    return info

def add_ubiq(info):
    # left, right 13
    gene, aaref = info['genename'], info['aaref']
    info['ubiquitination'] = 0
    if gene in ubiquitination:
        aapos = map(int, info['aapos'].split(';'))
        new_pos = list(aapos)
        for pos in aapos: # pos is 1 based            
            for i in range(1, 14): # left, right flanking of 13 bases
                new_pos.append(pos + i)
                new_pos.append(pos - i)
        for pos in new_pos:
            if pos in ubiquitination[gene]: #if phosphorylation[gene][pos]['AA'] == aaref:
                info['ubiquitination'] = ubiquitination[gene][pos]
                break
    return info

def add_interface(info):
    gene, aaref = info['genename'], info['aaref']
    info['interface'] = 0
    if gene in interface:
        aapos = info['aapos'].split(';')
        for pos in aapos:
            pos = int(pos)
            # AA_seq start from 0
            protein_length = len(AA_seq[gene])
            if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                if pos in interface[gene]:
                    info['interface'] = 1
    return info

def add_ASA(info):
    # add Accessible Surface Areas(ASA) score 
    gene, aaref = info['genename'], info['aaref']
    info['ASA'] = 0
    if gene in ASA:
        aapos = info['aapos'].split(';')
        for pos in aapos:
            pos = int(pos)
            # AA_seq start from 0
            protein_length = len(AA_seq[gene])
            if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                if pos in ASA[gene]:
                    info['ASA'] = ASA[gene][pos]
    return info

                                
def choose_variants(info, prefix, gnomad_AF):
    include_variants = False
    if prefix in '.All.':
        include_variants = True
    elif prefix in '.HIS.': # HIS genes
        if float(info['pli']) >= 0.5: 
            include_variants = True
    elif prefix in '.HS.': # HS genes are pli < 0.5 and pli unknown
        if float(info['pli']) < 0.5: # include variants with missing pli(-1)
            include_variants = True  
    if float(info['gnomad_exome']) > gnomad_AF:
        include_variants = False
    return include_variants
    

In [3]:
def sel_add_features(fin, w, 
                     wgsa_feat, add_feat, extra_feat, 
                     target_value, prefix, 
                     gnomad_AF=1, write_head=True):
    """ function that selected colums from wgsa, add some other feathers 
        set . into 0
    """
    with open(fin, 'rU') as f:
        positive, negative = 0, 0
        r = csv.reader(f,delimiter=',')
        head = r.next()
        feat_all = wgsa_feat + add_feat + extra_feat
        feat = []

        # set feature orders, order_feat first, then all other features
        for f in order_feat:
            if f in feat_all:
                feat.append(f)
        for f in feat_all:
            if f not in feat:
                feat.append(f)
        if write_head:
            w.writerow(feat)

        for line in r:
            info = dict(zip(head, line))
            aaref, aaalt, aapos = info['aaref'], info['aaalt'], info['aapos']                
            var_id = '_'.join([info['hg19_chr'], info['hg19_pos(1-based)'], info['ref'], info['alt']])
            info['var_id'] = var_id
            info['genename'] = info['genename'].split(';')[0] # pick the first gene if in many genes
            
            if var_id in exclude_var: continue

            # exclude nonsense variants and syn
            if aaref not in {'X', '.'} and aaalt not in {'X', '.'}: 
                # reformat wsga feat, missing value filled with 0, will -1 be better?
                for c in wgsa_feat:
                    if info[c] == '.':
                        if c in {'ExAC_AF', '1000Gp3_AF'}:
                            info[c] = 0
                        else:
                            info[c] = -1
                            
                    else:
                        try:
                            info[c] = float(info[c])
                        except Exception, e:
                            print str(e)
                            print e
                            print info[c]
                        
                # set some default value for extra_feat
                for c in extra_feat:
                    info[c] = info.get(c, 'NA')
                
                
                # deal with added features. 
                # update SUMO/phospho scores
                info = add_phospho(info)
                info = add_SUMO(info)
                info = add_interface(info)
                info = add_ASA(info)
                info = add_ubiq(info)
                info['blosum62'] = matrix_score(aaref, aaalt, 'blosum62') 
                info['pam250'] = matrix_score(aaref, aaalt, 'pam250')
                # gene specific feathers
                info['complex_CORUM'] = 0
                if info['genename'] in complex_CORUM:
                    info['complex_CORUM'] = 1
                info['preppi_counts'] = preppi.get(info['genename'], 0)
                info = add_secondary(info)
                info = add_gene_metric(info)
                info = add_gc_content(info)
                info = add_s_het(info)
                info = add_BioPlex(info)
                info = add_MPC(info)
                info = add_REVEL(info)
                info = add_target(info, target_value)
                info = add_gnomad(info) # add gnomad WGS and WES at the same time, missing keys get zero AF
               
                # choose variants in HIS or HS
                include_variants = choose_variants(info, prefix, gnomad_AF)
                if include_variants:
                    if info['target'] == 1:
                        positive += 1
                    elif info['target'] == 0:
                        negative += 1
                    else:
                        print info['target'], 'sth wrong'
                    w.writerow([info[c] for c in feat])
    print '{} pos, {} neg'.format(positive, negative)

In [4]:
# variants excluded for training
exclude_var = set()
with open('../data/excluded_variants_gwas.txt') as f:
    for line in f:
        exclude_var.add(line.strip())  
with open('../data/input_data.exclude.txt') as f:
    for line in f:
        exclude_var.add(line.strip())

In [5]:
# Load protein related annotations ## these are dictionaries !!! not array
SUMO = np.load('../data/protein/SUMO.npy').item() # dict of gene dict, dict of pos 
phosphorylation = np.load('../data/protein/phosphorylation.npy').item() # same format as SUMO
ASA = np.load('../data/protein/ASA.npy').item() # dict of gene, of pos and its asa value
ubiquitination = np.load('../data/protein/ubiquitination.npy').item()# dict of gene, of pos and its ubi value

secondary = np.load('../data/protein/secondary.npy').item()# dict of gene, of pos and secondary value

AA_seq = np.load('../data/protein/AA_seq.npy').item() # gene to sequence
interface = np.load('../data/protein/interface.npy').item() # gene to positions
preppi = np.load('../data/protein/preppi.npy').item() # gene to value
BioPlex = np.load('../data/protein/BioPlex.npy').item()# gene to value
s_het = np.load('../data/gene/s_het.npy').item() # gene to value
prec = np.load('../data/gene/prec.npy').item() # gene to value
pli = np.load('../data/gene/pli.npy').item() # gene to value
lofz = np.load('../data/gene/lofz.npy').item() # gene to value
domino = np.load('../data/gene/domino.npy').item() # gene to value

complex_CORUM =  np.load('../data/protein/complex_CORUM.npy').item() # set of genes

gnomad_af = np.load('../data/training/gnomad_af.npy').item() # variant to value
gnomad_exome_af = np.load('../data/training/gnomad_exome_new.npy').item()# variant to value

In [6]:
# feature order from correlation cluster
order_feat = [u'MutationAssessor_score_rankscore', u'VEST3_rankscore', u'Polyphen2_HDIV_rankscore',
 u'Polyphen2_HVAR_rankscore', u'SIFT_converted_rankscore', u'PROVEAN_converted_rankscore',
 u'MetaSVM_rankscore',u'MetaLR_rankscore', u'FATHMM_converted_rankscore', u'M-CAP_rankscore',
 u'GenoCanyon_score_rankscore', u'LRT_converted_rankscore', u'Eigen-PC-raw_rankscore',
 u'Eigen-phred', u'Eigen-PC-phred', u'DANN_rankscore', u'CADD_phred', u'CADD_raw_rankscore',
 u'phyloP20way_mammalian_rankscore', u'GERP++_RS_rankscore', u'SiPhy_29way_logOdds_rankscore',
 u'phastCons100way_vertebrate_rankscore', u'fathmm-MKL_coding_rankscore', u'phyloP100way_vertebrate_rankscore',
 u'MutationTaster_converted_rankscore', u'phastCons20way_mammalian_rankscore', u'GM12878_fitCons_score_rankscore',
 u'HUVEC_fitCons_score_rankscore', u'integrated_fitCons_score_rankscore',u'H1-hESC_fitCons_score_rankscore', 
 u'blosum62', u'pam250', u'SUMO_diff', u'SUMO_score', u'SUMO_cutoff', u'phospho_cutoff', u'phospho_score',
 u'phospho_diff', u'lofz', u'prec', u'pli', u'domino', 
 u's_het', u's_het_log', u'secondary_E', u'secondary_H', u'complex_CORUM', u'preppi_counts', 'haiyuan_interface', 
 u'1000Gp3_AF', u'ExAC_AF', 'gnomad', 'gnomad_exome', u'ASA', u'secondary_C', u'gc_content', u'interface', u'ubiquitination']

In [7]:
# add feathers from WGSA and other inputs, some of them need to be excluded in future training
rank_score_cols = ['SIFT_converted_rankscore', 'Polyphen2_HDIV_rankscore', 'Polyphen2_HVAR_rankscore', 
 'LRT_converted_rankscore', 'MutationTaster_converted_rankscore', 'MutationAssessor_score_rankscore', 
 'FATHMM_converted_rankscore', 'PROVEAN_converted_rankscore', 'VEST3_rankscore', 
 'MetaSVM_rankscore', 'MetaLR_rankscore', 'M-CAP_rankscore', 
 'CADD_raw_rankscore', 'DANN_rankscore', 'fathmm-MKL_coding_rankscore', 
 'Eigen-PC-raw_rankscore', 'GenoCanyon_score_rankscore', 'integrated_fitCons_score_rankscore', 
 'GM12878_fitCons_score_rankscore', 'H1-hESC_fitCons_score_rankscore', 
 'HUVEC_fitCons_score_rankscore', 'GERP++_RS_rankscore', 
 'phyloP100way_vertebrate_rankscore', 'phyloP20way_mammalian_rankscore', 
 'phastCons100way_vertebrate_rankscore', 'phastCons20way_mammalian_rankscore', 
 'SiPhy_29way_logOdds_rankscore']

wgsa_feat = ['1000Gp3_AF', 'ExAC_AF', 'CADD_phred', 'Eigen-phred', 'Eigen-PC-phred', 'RVIS']
wgsa_feat = wgsa_feat + rank_score_cols

add_feat =  ['blosum62', 'pam250', 'SUMO_score', 'SUMO_cutoff', 'SUMO_diff',
             'phospho_score', 'phospho_cutoff','phospho_diff', 'interface',
             'ASA', 'pli', 'lofz', 'domino', 'complex_CORUM', 'preppi_counts',
             'secondary_H', 'secondary_C', 'secondary_E', 'ubiquitination',
             'prec', 's_het_log', 'gc_content', 'gnomad', 'gnomad_exome', 'BioPlex',
             'obs_exp', 'mis_badness', 'MPC', 'REVEL', 
             'target'] 
              
# feathers used for future info
extra_feat = ['hg19_chr', 'hg19_pos(1-based)', 
              'ref', 'alt', 'aaref','aaalt','category', 'source','INFO', 'disease', 
              'genename', 'Ensembl_transcriptid', 'var_id']


# add anno to training files/testing sets

In [ ]:
prefixs = ['.HIS.', '.HS.']
for prefix in prefixs:
    # positive training files
    fins = ['../data/training/HGMD_DM_missense_norecceive.rare.csv',
            '../data/training/MPC_train.rare.csv' , # only 400 HIS variants
            '../data/training/clinvar_pathogenic_1-4star.rare.csv']
    
    fouts = [f.split('.csv')[0] + prefix + 'reformat.csv' for f in fins]
    for fin, fout in zip(fins, fouts):
        print fout
        with open(fout, 'w') as fw:
            w = csv.writer(fw)
            target_value = 1
            sel_add_features(fin, w, 
                            wgsa_feat, add_feat, extra_feat, 
                            target_value, prefix, 
                            gnomad_AF = 0.0001, write_head=True)
            
    # negative training files
    fins = ['../data/training/DiscovEHR_missense_sel.rare.csv',
            '../data/training/DiscovEHR_rare_missense_30000.csv',
            '../data/training/CADD_neg_train.anno.rare.csv']

    fouts = [f.split('.csv')[0] + prefix + 'reformat.csv' for f in fins]
    for fin, fout in zip(fins, fouts):
        print fout
        with open(fout, 'w') as fw:
            w = csv.writer(fw)
            target_value = 0
            sel_add_features(fin, w, 
                            wgsa_feat, add_feat, extra_feat, 
                            target_value, prefix, 
                            gnomad_AF = 0.0001, write_head=True)

In [ ]:
# metaSVM and other testing
prefixs = ['.HIS.', '.HS.']
for prefix in prefixs:
    fins = ['../data/metaSVM/metaSVM_train.anno.rare.csv', 
            '../data/metaSVM/metaSVM_test1.anno.rare.csv', 
            '../data/metaSVM/metaSVM_test2.anno.rare.csv', 
            '../data/metaSVM/metaSVM_test3.anno.rare.csv', 
            '../data/metaSVM/metaSVM_addtest1.anno.rare.csv', 
            '../data/metaSVM/metaSVM_addtest2.anno.rare.csv',
            '../data/cancer_hotspots/cancer_sel.csv',
            '../data/cancer_hotspots/hotspot_paper_randomsel.anno.rare.final.csv']
    
    fouts = [f.split('.csv')[0] + prefix + 'reformat.csv' for f in fins]
    for fin, fout in zip(fins, fouts):
        with open(fout, 'w') as fw:
            print fout
            w = csv.writer(fw)
            target_value = 'NA'
            sel_add_features(fin, w, 
                            wgsa_feat, add_feat, extra_feat, 
                            target_value, prefix,
                            gnomad_AF = 0.0001,
                            write_head=True)

# add annotation to de novo case and control sets

In [9]:
# de novo variants
prefixs = ['.HIS.', '.HS.']#, '.All.']
for prefix in prefixs:
    # control
    fcontrol = ['../data/case_control/control_1911.anno.rare.final.csv',
                '../data/case_control/control_MarkDaly.anno.rare.final.csv']
    
    # case
    fcase = ['../data/case_control/asd.anno.rare.final.csv', 
             '../data/case_control/DDD_new_0.2.anno.rare.final.csv',
             '../data/case_control/chd_yale.anno.rare.final.csv',
             '../data/case_control/CHD_DDD_YALE.anno.rare.final.csv',
             '../data/case_control/case_MarkDaly.anno.rare.final.csv',
             '../data/case_control/spark.anno.rare.final.csv',
             '../data/case_control/SSC_Xueya.anno.rare.final.csv',
             '../data/case_control/spark0315.anno.rare.csv']

    
    for fins, target in zip([fcase, fcontrol], [1, 0]):
        fouts = [f.split('.csv')[0] + prefix + 'reformat.csv' for f in fins]
        for fin, fout in zip(fins, fouts):
            with open(fout, 'w') as fw:
                print fin.split('/')[-1], fout, target
                w = csv.writer(fw)
                target_value = target
                sel_add_features(fin, w, 
                                wgsa_feat, add_feat, extra_feat, 
                                target_value, prefix,
                                gnomad_AF=1.0, # filter in future analysis
                                write_head=True)

spark0315.anno.rare.final.csv ../data/case_control/spark0315.anno.rare.final.HIS.reformat.csv 1
149 pos, 0 neg
spark0315.anno.rare.final.csv ../data/case_control/spark0315.anno.rare.final.HS.reformat.csv 1
244 pos, 0 neg


In [ ]:
prefixs = ['.HIS.', '.HS.']
for prefix in prefixs:

    # negative
    fins = ['../data/training/DiscovEHR_missense.rare.csv']

    fouts = [f.split('.csv')[0] + prefix + 'reformat.csv' for f in fins]
    for fin, fout in zip(fins, fouts):
        print fout
        with open(fout, 'w') as fw:
            w = csv.writer(fw)
            target_value = 0
            sel_add_features(fin, w, 
                            wgsa_feat, add_feat, extra_feat, 
                            target_value, prefix, 
                            gnomad_AF = 0.0001, 
                            write_head=True)

# process all missense and all cancer

In [ ]:
# all cancer hostspot and other files, keep all 1% variants 

prefix = '.All.'
fins = ['/data/hq2130/large_files/rare_missense_id.anno.rare.csv']
fouts = [f.split('.csv')[0] + prefix + 'reformat.csv' for f in fins]
for fin, fout in zip(fins, fouts):
    with open(fout, 'w') as fw:
        w = csv.writer(fw)
        target_value = 0
        sel_add_features(fin, w, wgsa_feat,
                         add_feat, extra_feat, 
                         target_value, prefix, 
                         gnomad_AF = 0.01, write_head=True)

print "done" 